# Acquisition Models for MR, PET and CT
This demonstration shows how to set-up and use SIRF acquisition models for different modalities.

This demo is a jupyter notebook, i.e. intended to be run step by step.
You could export it as a Python file and run it one go, but that might
make little sense as the figures are not labelled.


Author: Christoph Kolbitsch
First version: 22nd of April 2021  

CCP PETMR Synergistic Image Reconstruction Framework (SIRF).  
Copyright 2015 - 2017 Rutherford Appleton Laboratory STFC.  
Copyright 2015 - 2019 University College London.   
Copyright 2021 Physikalisch-Technische Bundesanstalt.

This is software developed for the Collaborative Computational
Project in Positron Emission Tomography and Magnetic Resonance imaging
(http://www.ccppetmr.ac.uk/).

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Initial set-up

In [ ]:
# Make sure figures appears inline and animations works
%matplotlib notebook

In [ ]:
# Make sure everything is installed that we need
!pip install brainweb nibabel --user

In [ ]:
# Initial imports etc
import numpy
import matplotlib.pyplot as plt

import os
import sys
import shutil
import brainweb
from tqdm.auto import tqdm

# Import MR, PET and CT functionality
import sirf.Gadgetron as mr
import sirf.STIR as pet
import cil.framework as ct

from sirf.Utilities import examples_data_path
#from ccpi.astra.operators import AstraProjectorSimple

# Utilities

In [ ]:
# First define some handy function definitions
# To make subsequent code cleaner, we have a few functions here. You can ignore
# ignore them when you first see this demo.
# They have (minimal) documentation using Python docstrings such that you 
# can do for instance "help(subplot_)"
#

def subplot_(idx,vol,title,clims=None,cmap="viridis"):
    """Customized version of subplot"""
    plt.subplot(*idx)
    plt.imshow(vol,cmap=cmap)
    if not clims is None:
        plt.clim(clims)
    plt.colorbar()
    plt.title(title)
    plt.axis("off")

def crop_and_fill(templ_im, vol):
    """Crop volumetric image data and replace image content in template image object"""
    # Get size of template image and crop
    #idim = templ_im.dimensions() # TODO
    idim_orig = templ_im.as_array().shape
    idim = (1,)*(3-len(idim_orig)) + idim_orig
    offset = (numpy.array(vol.shape) - numpy.array(idim)) // 2
    vol = vol[offset[0]:offset[0]+idim[0], offset[1]:offset[1]+idim[1], offset[2]:offset[2]+idim[2]]
    # Make a copy of the template to ensure we do not overwrite it
    templ_im_out = templ_im.copy()
    # Fill image content 
    templ_im_out.fill(numpy.reshape(vol, idim_orig))
    return(templ_im_out)

# Get brainweb data

We will download and use data from the brainweb. We will use a FDG image for PET. MR usually provides qualitative images with an image contrast proportional to difference in T1, T2 or T2* depending on the sequence parameters. Nevertheless, we will make our life easy, by directly using the T1 map provided by the brainweb for MR.

In [ ]:
fname, url= sorted(brainweb.utils.LINKS.items())[0]
files = brainweb.get_file(fname, url, ".")
data = brainweb.load_file(fname)

brainweb.seed(1337)

In [ ]:
for f in tqdm([fname], desc="mMR ground truths", unit="subject"):
    vol = brainweb.get_mmr_fromfile(f, petNoise=1, t1Noise=0.75, t2Noise=0.75, petSigma=1, t1Sigma=1, t2Sigma=1)

In [ ]:
FDG_arr  = vol['PET']
T1_arr   = vol['T1']
uMap_arr = vol['uMap']

In [ ]:
# Display it
plt.figure();
slice_show = FDG_arr.shape[0]//2
subplot_([1,3,1], FDG_arr[slice_show, 100:-100, 100:-100], 'FDG', cmap="hot")
subplot_([1,3,2], T1_arr[slice_show, 100:-100, 100:-100], 'T1', cmap="Greys_r")
subplot_([1,3,3], uMap_arr[slice_show, 100:-100, 100:-100], 'uMap', cmap="bone")

# Acquisition Models

In SIRF an `AcquisitionModel` basically contains everything we need to know in order to describe what happens when we go from the imaged object to the acquired raw data (`AcquisitionData`) and then to the reconstructed image (`ImageData`). What we actually need to know depends strongly on the modality we are looking at. 

Here are some examples of modality specific information:  

  * __PET__: scanner geometry, attenuation map, detector efficiency,...  
  * __CT__: scanner geometry  
  * __MR__: k-space sampling pattern, coil sensitivity information,...

and then there is information which is independent of the modality such as field-of-view or image resolution.

For __PET__ and __MR__ a lot of this information is already in the raw data. Because it would be quite a lot of work to enter all the necessary information by hand and then checking it is consistent, we create `AcquisitionModel` objects from `AcquisitionData` objects. The `AcquisitionData` only serves as a template and both its actual image and raw data content can be (and in this exercise will be) replaced. For __CT__ we will create an acquisition model from scratch, i.e. we will define the scanner geometry, image dimensions and image voxels sizes and so by hand.

So let's get started with the __MR__

## MR

For the MR we basically need the following:

  1. create an MR `AcquisitionData` object from a raw data file
  2. calculate the coil sensitivity maps (csm, for more information on that please see the notebook MR/c_coil_combindation.ipynb). These csm are a derived class from MR `ImageData` and therefore can also be used as a template for the image data we want to reconstruct
  3. then we will set up the MR `AcquisitionModel`

In [ ]:
# 1. create MR AcquisitionData
mr_acq = mr.AcquisitionData(examples_data_path('MR') + '/grappa2_1rep.h5')

In [ ]:
# 2. calculate CSM
preprocessed_data = mr.preprocess_acquisition_data(mr_acq)

csm = mr.CoilSensitivityData()
csm.smoothness = 50
csm.calculate(preprocessed_data)

im_mr = csm.copy()

In [ ]:
# 3. create AcquisitionModel
acq_mod_mr = mr.AcquisitionModel(preprocessed_data, im_mr)

# Supply csm to the acquisition model 
acq_mod_mr.set_coil_sensitivity_maps(csm)

# Apply acquisition models

Now we have got an MR image object and can fill it with the brainweb data. The dimensions won't fit, but we will simply crop the image.

In [ ]:
im_mr = crop_and_fill(im_mr, T1_arr)
plt.figure();
subplot_([1,1,1], numpy.abs(im_mr.as_array())[im_mr.dimensions()[0]//2, :, :], 'MR', cmap="Greys_r")

# CT

In [ ]:
#%% Use the 'ct' prefix for all CIL-based SIRF functions.
# This is done here to explicitly differentiate between SIRF ct functions and 
# anything else.
import cil.framework as ct #import ImageGeometry, AcquisitionGeometry

In [ ]:
#%% Create a template CT acquisition geometry
N = 120
angles = numpy.linspace(0, 360, 50, True, dtype=numpy.float32)
offset = 0.4
channels = 1
ag = ct.AcquisitionGeometry.create_Cone3D((offset,-100, 0), (offset,100,0))
ag.set_panel((N,N-2))
ag.set_channels(channels)
ag.set_angles(angles, angle_unit=ct.AcquisitionGeometry.DEGREE)

In [ ]:
# Now we can create a template CT image object
ig = ag.get_ImageGeometry()
im_ct = ig.allocate(None)

Now we have got an CT image object and can fill it with the brainweb data. The dimensions won't fit, but we will simply crop the image.

In [ ]:
im_ct = crop_and_fill(im_ct, uMap_arr)

plt.figure();
subplot_([1,1,1], im_ct.as_array()[im_ct.as_array().shape[0]//2, :, :], 'CT', cmap="bone")

# PET

In [ ]:
#%% Use the 'pet' prefix for all STIR-based SIRF functions.
# This is done here to explicitly differentiate between SIRF pet functions and 
# anything else.
import sirf.STIR as pet

In [ ]:
# We'll need a template sinogram
templ_sino = pet.AcquisitionData(examples_data_path('PET') + "/thorax_single_slice/template_sinogram.hs")

In [ ]:
# Now we can create a template PET image object
im_pet = pet.ImageData(templ_sino)

Now we have got an PET image object and can fill it with the brainweb data. The dimensions won't fit, but we will simply crop the image.

In [ ]:
im_pet = crop_and_fill(im_pet, FDG_arr)

plt.figure();
subplot_([1,1,1], im_pet.as_array()[im_pet.dimensions()[0]//2, :, :], 'PET', cmap="hot")

# Create acquisition models

In [ ]:
#%% Create an acquisition model for MR

# First we need the coil sensitivity information
csm = mr.CoilSensitivityData()
csm.smoothness = 50
csm.calculate(preprocessed_data)

# create MR acquisition model
acq_mod_mr = mr.AcquisitionModel(preprocessed_data, im_mr)

# to supply coil info to the acquisition model we use the dedicated method
acq_mod_mr.set_coil_sensitivity_maps(csm)

In [ ]:
#%% Create an acquisition model for PET

# First we need a uMap
uMap = crop_and_fill(im_pet, uMap_arr)
#uMap = crop_and_fill(im_pet, numpy.ones((300,300,300)))

# create PET acquisition model
acq_mod_pet = pet.AcquisitionModelUsingRayTracingMatrix()
acq_mod_pet.set_num_tangential_LORs(5)

if False:
    # Set up sensitivity due to attenuation
    asm_attn = pet.AcquisitionSensitivityModel(uMap, acq_mod_pet)
    asm_attn.set_up(templ_sino)
    bin_eff = pet.AcquisitionData(templ_sino)
    bin_eff.fill(1.0)

    # Apply attenuation
    asm_attn.unnormalise(bin_eff)
    asm_attn = pet.AcquisitionSensitivityModel(bin_eff)

    acq_mod_pet.set_acquisition_sensitivity(asm_attn)

acq_mod_pet.set_up(templ_sino, uMap)

In [ ]:
#%% Create an acquisition model for CT

#acq_mod_ct = AstraProjectorSimple(ig, ag, 'cpu')

# Apply acquisition models

In [ ]:
#%% Now we have got all the acquisition models and can apply the forward model for

# PET
raw_pet = acq_mod_pet.forward(im_pet)

# MR
raw_mr = acq_mod_mr.forward(im_mr)

# CT
#raw_ct = acq_mod_ct.forward(im_ct)

# and apply the backward model

# PET
rec_pet = acq_mod_pet.backward(raw_pet)

# MR
rec_mr = acq_mod_mr.backward(raw_mr)

# CT

# Display results

In [ ]:
#raw_dat_ct.as_array().shape

In [ ]:
#rec_pet.as_array().shape

In [ ]:
plt.figure();
# Raw data
subplot_([2,3,1], numpy.log(numpy.abs(raw_mr.as_array()[:, raw_mr.dimensions()[1]//2, :])), 'MR raw', cmap="viridis")
subplot_([2,3,2], raw_pet.as_array()[0, raw_pet.dimensions()[1]//2, :, :], 'PET raw', cmap="viridis")

# Rec data
subplot_([2,3,4], numpy.abs(rec_mr.as_array()[rec_mr.dimensions()[0]//2, :, :]), 'MR rec', cmap="Greys_r")
subplot_([2,3,5], rec_pet.as_array()[rec_pet.dimensions()[0]//2, :, :], 'PET rec', cmap="viridis")